In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
This code demonstrates how to use dedupe with a comma separated values
(CSV) file. All operations are performed in memory, so will run very
quickly on datasets up to ~10,000 rows.

We start with a CSV file containing our messy data. In this example,
it is listings of early childhood education centers in Chicago
compiled from several different sources.

The output will be a CSV with our clustered results.

For larger datasets, see our [mysql_example](mysql_example.html)
"""

import os
import csv
import re
import logging
import optparse
import pandas as pd

import dedupe
from unidecode import unidecode

In [2]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    # If data is missing, indicate that by setting the value to `None`
    if not column:
        column = None
    return column


def readData(filename):
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """

    data_d = {}
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d

In [3]:
# ## Setup
retrain = input('Do you want to add on to your training (y/n). If you wanted to start over, delete your .json file')
isretrain = True if retrain == 'y' else False

if isretrain == True:
    try:
        os.remove('csv_example_learned_settings')
    except:
        print('Your settings file appears to not have existed.')

#input_file = 'csv_example_messy_input.csv'
input_file = '../../Data/Outputs_Cleanup/Part_d/partD_dedupe_input_drugs.csv'
output_file = '../../Data/Outputs_Cleanup/Part_d/dedupe_output/MedicareD_drug_deduplicated.csv'
settings_file = '../../Data/Outputs_Cleanup/Part_d/dedupe_output/csv_example_learned_settings'
training_file = '../../Data/Outputs_Cleanup/Part_d/dedupe_output/csv_example_training.json'

print('importing data ...')
data_d = readData(input_file)

# If a settings file already exists, we'll just load that and skip training
if os.path.exists(settings_file):
    print('reading from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    # ## Training

    # Define the fields dedupe will pay attention to
    fields = [
        {'field': 'brand_name', 'type': 'String'},
        #{'field': 'Address', 'type': 'String'},
        #{'field': 'Zip', 'type': 'Exact', 'has missing': True},
        #{'field': 'Phone', 'type': 'String', 'has missing': True},
        ]

    # Create a new deduper object and pass our data model to it.
    deduper = dedupe.Dedupe(fields)

    # If we have training data saved from a previous run of dedupe,
    # look for it and load it in.
    # __Note:__ if you want to train from scratch, delete the training_file
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.prepare_training(data_d, f)
    else:
        deduper.prepare_training(data_d)

    # ## Active learning
    # Dedupe will find the next pair of records
    # it is least certain about and ask you to label them as duplicates
    # or not.
    # use 'y', 'n' and 'u' keys to flag duplicates
    # press 'f' when you are finished
    print('starting active labeling...')

    dedupe.console_label(deduper)

    # Using the examples we just labeled, train the deduper and learn
    # blocking predicates
    deduper.train()

    # When finished, save our training to disk
    with open(training_file, 'w') as tf:
        deduper.write_training(tf)

    # Save our weights and predicates to disk.  If the settings file
    # exists, we will skip all the training and learning next time we run
    # this file.
    with open(settings_file, 'wb') as sf:
        deduper.write_settings(sf)

# ## Clustering

# `partition` will return sets of records that dedupe
# believes are all referring to the same entity.

print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

# ## Writing Results

# Write our original data back out to a CSV with a new column called
# 'Cluster ID' which indicates which records refer to each other.

cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

with open(output_file, 'w') as f_output, open(input_file) as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['Id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

Do you want to add on to your training (y/n). If you wanted to start over, delete your .json file n


importing data ...


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (fingerprint, brand_name)
brand_name : easy comfort pen needle

brand_name : easy comfort pen needles

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


starting active labeling...


 y


...

 y


brand_name : methylphenidate er

brand_name : methylphenidate er la

13/10 positive, 16/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


 f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:TfidfNGramCanopyPredicate: (0.8, brand_name)
INFO:dedupe.training:SimplePredicate: (oneGramFingerprint, brand_name)
INFO:dedupe.training:SimplePredicate: (commonThreeTokens, brand_name)
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.22942021846123362
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(TfidfNGramCanopyPredicate: (0.8, brand_name), SimplePredicate: (sortedAcronym, brand_name), TfidfNGramCanopyPredicate: (0.2, brand_name))
INFO:dedupe.training:(LevenshteinCanopyPredicate: (1, brand_name), SimplePredicate: (firstTwoTokensPredicate, brand_name), TfidfNGramCanopyPredicate: (0.6, brand_name))
INFO:dedupe.training:(SimplePredicate: (firstTwoTokensPredicate, brand_name), TfidfTextCanopyPredicate: (0.2, brand_name), SimplePredicate: (oneGramFingerprint, brand_name))
INFO:dedupe.training:(SimplePredicate: 

clustering...
# duplicate sets 2303


In [4]:
def pick_brand(brands):
    pick = None
    pick_len = 0
    for idx, brand in enumerate(brands):
        if idx == 0:
            pick = brand
            pick_len = len(brand)
            continue
        if len(brand) > pick_len:
            pick = brand
            pick_len = len(brand)
    return pick

df = pd.read_csv('MedicareD_drug_deduplicated.csv')

In [5]:
df.head()

,Cluster ID,confidence_score,Unnamed: 2,brand_name,Id
0,17,1.0,0,1st Tier Unifine Pentips Plus,0
1,18,1.0,1,Abacavir,1
2,19,1.0,2,Abacavir-Lamivudine,2
3,20,1.0,3,Abacavir-Lamivudine-Zidovudine,3
4,21,1.0,4,Abilify,4


In [8]:
df_cluster = pd.DataFrame(df.groupby('Cluster ID')['brand_name'].apply(list))
df_cluster['Id'] = df.groupby('Cluster ID')['Id'].apply(list)
df_cluster['brand_name_picked'] = df_cluster.brand_name.apply(pick_brand)
df_cluster.columns = ['brand_names', 'MedD_drug_Id', 'brand_name']
df_cluster.to_csv('../../Data/Outputs_Cleanup/Part_d/dedupe_output/MedicareD_Drug_deduplicated_singlebrand.csv')